##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# tff의 ClientData로 작업하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/working_with_client_data"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.36.0/docs/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.36.0/docs/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/federated/docs/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

클라이언트(예: 사용자)에 의해 키가 지정된 데이터세트의 개념은 TFF에서 모델링된 페더레이션 계산에 필수적입니다. TFF는 [`tff.simulation.datasets.ClientData`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData) 인터페이스를 제공하여 이 개념을 추상화하고 TFF가 호스팅하는 데이터세트([stackoverflow](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/stackoverflow), [shakespeare](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/shakespeare), [emnist](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/emnist), [cifar100](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/cifar100) 및 [gldv2](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/gldv2))는 모두 이 인터페이스를 구현합니다.

자신의 데이터세트로 페더레이션 학습을 수행하는 경우, TFF는 `ClientData` 인터페이스를 구현하거나 TFF의 도우미 함수 중 하나를 사용하여 디스크의 데이터를 나타내는 `ClientData`를 생성할 것을 강력히 권장합니다(예: [`tff.simulation.datasets.ClientData.from_clients_and_fn`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData#from_clients_and_fn)).

TFF의 종단 간 예제 대부분은 `ClientData` 객체로 시작하므로, 사용자 정의 데이터세트와 함께 `ClientData` 인터페이스를 구현하면 TFF로 작성된 기존 코드를 통해 더 쉽게 spelunk를 만들 수 있습니다. 또한 `ClientData`가 구성하는 `tf.data.Datasets`는 `numpy` 배열 구조를 생성하기 위해 직접 반복될 수 있으므로 `ClientData` 객체는 TFF로 이동하기 전에 Python 기반 ML 프레임워크와 함께 사용할 수 있습니다.

시뮬레이션을 여러 시스템으로 확장하거나 배포하려는 경우 편리성을 높일 수 있는 몇 가지 패턴이 있습니다. 아래에서는 `ClientData` 및 TFF를 사용하여 소규모 반복에서 대규모 실험, 프로덕션 배포 환경까지의 확장을 최대한 원활하게 만드는 몇 가지 방법을 살펴보겠습니다.

## ClientData를 TFF로 전달하려면 어떤 패턴을 사용해야 합니까?

TFF의 `ClientData`를 사용하는 두 가지 방법에 대해 자세히 설명합니다. 아래 두 범주 중 하나에 해당한다면 분명히 둘 중 선호하는 하나가 있을 것입니다. 그렇지 않은 경우 보다 미묘한 선택을 하기 위해 각각의 장단점을 보다 자세히 이해해야 할 수 있습니다.

- 로컬 시스템에서 가능한 한 빠르게 반복하고 싶습니다. TFF의 분산 런타임을 쉽게 활용할 필요는 없습니다.

    - `tf.data.Datasets`를 TFF에 직접 전달하려고 합니다.
    - 이를 통해 `tf.data.Dataset` 객체를 사용하여 명령적으로 프로그래밍하고 임의로 처리할 수 있습니다.
    - 아래 옵션보다 더 많은 유연성을 제공합니다. 클라이언트에 논리를 푸시하려면 이 논리를 직렬화할 수 있어야 합니다.

- TFF의 원격 런타임에서 페더레이션 계산을 실행하고 싶거나 곧 그렇게 할 계획입니다.

    - 이 경우 데이터세트 구성 및 전처리를 클라이언트에 매핑하려고 합니다.
    - 그 결과 단순히 `client_ids` 목록을 페더레이션 계산에 직접 전달하게 됩니다.

- 클라이언트에 데이터세트 구성과 전처리를 푸시하면 직렬화의 병목 현상이 방지되고 수백에서 수천 개의 클라이언트 규모에서 성능이 크게 향상됩니다.

In [ ]:
#@title Set up open-source environment
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow_federated
!pip install --quiet --upgrade nest_asyncio

import nest_asyncio
nest_asyncio.apply()

In [1]:
#@title Import packages
import collections
import time

import tensorflow as tf
import tensorflow_federated as tff

## ClientData 객체 조작하기

TFF의 EMNIST `ClientData`를 로드하고 탐색하는 것으로 시작하겠습니다.


In [2]:
client_data, _ = tff.simulation.datasets.emnist.load_data()

첫 번째 데이터세트를 검사하면 `ClientData`에 어떤 유형의 예제가 있는지 알 수 있습니다.

In [3]:
first_client_id = client_data.client_ids[0]
first_client_dataset = client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)
# This information is also available as a `ClientData` property:
assert client_data.element_type_structure == first_client_dataset.element_spec

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])


데이터세트는 `pixels`과 `label` 키가 있는 `collections.OrderedDict` 객체를 생성합니다. 여기서 픽셀은 `[28, 28]` 형상을 가진 텐서입니다. 입력을 평면화하여 `[784]` 형상으로 만들고 싶다고 가정합니다. 이를 수행할 수 있는 한 가지 가능한 방법은 전처리 함수를 `ClientData` 객체에 적용하는 것입니다.

In [4]:
def preprocess_dataset(dataset):
  """Create batches of 5 examples, and limit to 3 batches."""

  def map_fn(input):
    return collections.OrderedDict(
        x=tf.reshape(input['pixels'], shape=(-1, 784)),
        y=tf.cast(tf.reshape(input['label'], shape=(-1, 1)), tf.int64),
    )

  return dataset.batch(5).map(
      map_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE).take(5)


preprocessed_client_data = client_data.preprocess(preprocess_dataset)

# Notice that we have both reshaped and renamed the elements of the ordered dict.
first_client_dataset = preprocessed_client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


이와 더불어, 예를 들어 셔플링과 같이 좀 더 복잡한(그리고 아마도 상태 저장인) 전처리를 수행해야 할 수도 있습니다.

In [5]:
def preprocess_and_shuffle(dataset):
  """Applies `preprocess_dataset` above and shuffles the result."""
  preprocessed = preprocess_dataset(dataset)
  return preprocessed.shuffle(buffer_size=5)

preprocessed_and_shuffled = client_data.preprocess(preprocess_and_shuffle)

# The type signature will remain the same, but the batches will be shuffled.
first_client_dataset = preprocessed_and_shuffled.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


## `tff.Computation`과 인터페이스 구성하기

이제 `ClientData` 객체로 몇 가지 기본적인 조작을 수행할 수 있으므로 `tff.Computation`에 데이터를 넣어볼 준비가 되었습니다. [페더레이션 평균화](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess)를 구현하는 <a><code>tff.templates.IterativeProcess</code></a>를 정의하고 데이터를 전달하는 다양한 방법을 탐색합니다.

In [6]:
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
  ])
  return tff.learning.from_keras_model(
      model,
      # Note: input spec is the _batched_ shape, and includes the 
      # label tensor which will be passed to the loss function. This model is
      # therefore configured to accept data _after_ it has been preprocessed.
      input_spec=collections.OrderedDict(
          x=tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
          y=tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  
trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01))

이 `IterativeProcess` 작업을 시작하기 전에 `ClientData`의 의미에 대해 한 가지 설명할 필요가 있을 것 같습니다. `ClientData` 객체는 페더레이션 훈련에 사용할 수 있는 *전체* 모집단을 나타내며, 이는 일반적으로 [프로덕션 FL 시스템의 실행 환경에서는 사용할 수 없고](https://arxiv.org/abs/1902.01046) 시뮬레이션에만 해당됩니다. `ClientData`는 실제로 사용자에게 페더레이션 컴퓨팅을 완전히 우회하고 [`ClientData.create_tf_dataset_from_all_clients`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData?hl=en#create_tf_dataset_from_all_clients)를 통해 평소와 같이 단순히 서버 측 모델을 훈련할 수 있게만 해줍니다.

TFF의 시뮬레이션 환경에서 연구자는 외부 루프를 완전히 제어할 수 있습니다. 특히 이것은 클라이언트 가용성, 클라이언트 드롭아웃 등의 고려 사항이 사용자 또는 Python 드라이버 스크립트에서 다루어져야 함을 의미합니다. 예를 들어 더 많은 데이터(및 이에 따라 더 오래 실행되는 로컬 계산)를 가진 사용자가 더 낮은 확률로 선택되도록 `ClientData's` `client_ids`에 대한 샘플링 분포를 조정하여 클라이언트 드롭아웃을 모델링할 수 있습니다.

그러나 실제 페더레이션 시스템에서는 모델 트레이너가 클라이언트를 명시적으로 선택할 수 없습니다. 클라이언트 선택은 페더레이션 계산을 실행하는 시스템에 위임됩니다.

### `tf.data.Datasets`를 TFF에 직접 전달하기

`ClientData`와 `IterativeProcess` 간의 인터페이스 구성하기 위한 한 가지 방법은 Python에서 `tf.data.Datasets`를 구성하고 이러한 데이터세트를 TFF에 전달하는 것입니다.

전처리된 `ClientData`를 사용하는 경우, 우리가 산출하는 데이터세트는 위에 정의된 모델에서 예상하는 적절한 유형입니다.

In [7]:
selected_client_ids = preprocessed_and_shuffled.client_ids[:10]

preprocessed_data_for_clients = [
    preprocessed_and_shuffled.create_tf_dataset_for_client(
        selected_client_ids[i]) for i in range(10)
]

state = trainer.initialize()
for _ in range(5):
  t1 = time.time()
  result = trainer.next(state, preprocessed_data_for_clients)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.934802532196045, round time 2.5420753955841064
loss 3.350963830947876, round time 0.45527172088623047
loss 3.1382687091827393, round time 0.47087883949279785
loss 3.0774152278900146, round time 0.4089682102203369
loss 2.9193594455718994, round time 0.3964221477508545


그러나 우리가 이 경로를 선택한다면 ***간단하게 다중 시스템 시뮬레이션으로 이동할 수 없을*** 것입니다. 로컬 TensorFlow 런타임에서 구성하는 데이터세트는 *주변 Python 환경에서 상태를 캡처*할 수 있으며 더 이상 사용할 수 없는 상태를 참조하려고 시도하면 직렬화 또는 역직렬화에 실패합니다. 이것은 예를 들어 TensorFlow의 `tensor_util.cc`에서 나타나는 불가사의한 오류로 극명히 나타날 수 있습니다.

```
Check failed: DT_VARIANT == input.dtype() (21 vs. 20)
```

### 여러 클라이언트에서 구성 및 전처리 매핑하기

이 문제를 방지하기 위해 TFF는 사용자가 데이터세트 인스턴스화 및 전처리를 *각 클라이언트에서 로컬로 발생*하는 것으로 간주하고 TFF의 헬퍼 또는 `federated_map`을 사용하여 각 클라이언트에서 이 전처리 코드를 명시적으로 실행할 것을 권장합니다.

개념적으로 이것을 선호하는 이유는 분명합니다. TFF의 로컬 런타임에서 전체 페더레이션 오케스트레이션이 단일 시스템에서 발생하기 때문에 클라이언트는 오로지 "우연히" 글로벌 Python 환경에 액세스할 수 있습니다. 이 시점에서 항상 직렬화 가능한 기능적 개념인 TFF의 크로스 플랫폼의 배경에도 유사한 사고 방식이 깔려 있다는 점에 주목할 필요가 있습니다.

TFF는 `ClientData's` 속성인 `dataset_computation`, `client_id`를 사용하고 연결된 `tf.data.Dataset`을 반환하는 `tff.Computation`을 통해 이러한 변경을 간단하게 만듭니다.

`preprocess`는 단순히 `dataset_computation`과 함께 작동합니다. 전처리된 `ClientData`의 `dataset_computation` 속성은 방금 정의한 전체 전처리 파이프라인을 통합합니다.

In [8]:
print('dataset computation without preprocessing:')
print(client_data.dataset_computation.type_signature)
print('\n')
print('dataset computation with preprocessing:')
print(preprocessed_and_shuffled.dataset_computation.type_signature)

dataset computation without preprocessing:
(string -> <label=int32,pixels=float32[28,28]>*)


dataset computation with preprocessing:
(string -> <x=float32[?,784],y=int64[?,1]>*)


우리는 `dataset_computation`을 호출하고 Python 런타임에서 즉시 실행 데이터세트를 받을 수 있지만 이 접근 방식이 가진 진정한 힘은 전역 즉시 실행 런타임에서 이러한 데이터세트를 구체화하는 것을 피하기 위해 반복 프로세스나 다른 계산으로 작성할 때 발휘됩니다. TFF는 정확히 이러한 목적을 위해 사용할 수 있는 헬퍼 함수 [`tff.simulation.compose_dataset_computation_with_iterative_process`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/compose_dataset_computation_with_iterative_process)를 제공합니다.

In [9]:
trainer_accepting_ids = tff.simulation.compose_dataset_computation_with_iterative_process(
    preprocessed_and_shuffled.dataset_computation, trainer)

이 `tff.templates.IterativeProcesses`와 위의 둘 모두 같은 방식으로 실행됩니다. 그러나 전자는 전처리된 클라이언트 데이터세트를 수용하고 후자는 클라이언트 ID를 나타내는 문자열을 수용하여 본문에서 데이터세트 구성과 전처리를 모두 처리합니다. 실제로 `state`는 둘 사이에 전달될 수 있습니다.

In [10]:
for _ in range(5):
  t1 = time.time()
  result = trainer_accepting_ids.next(state, selected_client_ids)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.6114611625671387, round time 1.4935951232910156
loss 2.612247943878174, round time 0.30751872062683105
loss 2.8368589878082275, round time 0.3043978214263916
loss 2.6863903999328613, round time 0.3107311725616455
loss 2.6816341876983643, round time 0.4325370788574219


### 많은 수의 클라이언트로 확장하기

`trainer_accepting_ids`는 TFF의 다중 시스템 런타임에서 즉시 사용할 수 있으며 `tf.data.Datasets` 및 컨트롤러의 구체화를 방지합니다(따라서 이를 직렬화하여 작업자에게 보냄).

이는 특히 많은 수의 클라이언트에서 분산 시뮬레이션의 속도를 크게 높이고 중간 집계를 사용하여 유사한 직렬화/역직렬화의 처리 부담을 덜어줍니다.


### 선택적 심층 작업: TFF에서 수동으로 전처리 논리 구성하기

TFF는 처음부터 구성성(compositionality)을 염두에 두고 설계되었습니다. TFF의 헬퍼가 방금 수행한 구성 유형은 사용자 입장에서 완전히 제어할 수 있습니다. 우리는 방금 정의한 전처리 계산을 트레이너 자신의 `next`로 아주 간단하게 수동으로 구성할 수 있었습니다.

In [11]:
selected_clients_type = tff.FederatedType(preprocessed_and_shuffled.dataset_computation.type_signature.parameter, tff.CLIENTS)

@tff.federated_computation(trainer.next.type_signature.parameter[0], selected_clients_type)
def new_next(server_state, selected_clients):
  preprocessed_data = tff.federated_map(preprocessed_and_shuffled.dataset_computation, selected_clients)
  return trainer.next(server_state, preprocessed_data)

manual_trainer_with_preprocessing = tff.templates.IterativeProcess(initialize_fn=trainer.initialize, next_fn=new_next)

사실, 이것은 우리가 사용한 헬퍼가 내부에서 수행하는 작업입니다(또한 적절한 유형 검사 및 조작도 수행). `preprocess_and_shuffle`을 `tff.Computation`으로 직렬화하고 `federated_map`을 전처리되지 않은 데이터세트를 구성하는 한 단계와 각 클라이언트에서 `preprocess_and_shuffle`을 실행하는 또 하나의 단계로 분해하여 동일한 논리를 약간 다르게 표현할 수도 있습니다.

보다 수동적인 이 경로를 취했을 때 TFF 헬퍼(모듈로 매개변수 이름)와 동일한 유형 서명을 사용하여 계산이 수행되는지 확인할 수 있습니다.

In [12]:
print(trainer_accepting_ids.next.type_signature)
print(manual_trainer_with_preprocessing.next.type_signature)

(<state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,client_data={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,metrics=<distributor=<>,client_work=<train=<sparse_categorical_accuracy=float32,loss=float32,num_examples=int64,num_batches=int64>>,aggregator=<mean_value=<>,mean_weight=<>>,finalizer=<>>@SERVER>)
(<server_state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,selected_clients={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggre